Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.


# LAB9

Write a local-search algorithm (eg. an EA) able to solve the _Problem_ instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

- Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
- Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

- Reviews will be assigned on Monday, December 4
- You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)


In [ ]:
from random import random, choices, choice, randint,shuffle
from tqdm import tqdm
import lab9_lib

In [ ]:
fitness = lab9_lib.make_problem(1)
for n in range(10):
    ind = choices([0, 1], k=10)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

1. what


# GA


In [ ]:
from concurrent.futures import ThreadPoolExecutor

class GA:
    def __init__(
        self,
        fitness,
        population_size=50,
        number_generations=100,
        mutation_rate=0.1,
        genome_size=1000,
        parent_selection_size=10,
    ):
        self.fitness = fitness
        self.population_size = population_size
        self.generations = number_generations
        self.population = [
            choices([0, 1], k=genome_size) for _ in range(population_size)
        ]
        self.mutation_rate = mutation_rate
        self.genome_size = genome_size
        self.parent_selection_size = parent_selection_size

    def mutate0(self, individual):
        return [bit ^ (random() < self.mutation_rate) for bit in individual]

    def crossover(self, parent1, parent2):
        point = int(random() * self.genome_size)
        # point = self.genome_size // 2
        return parent1[:point] + parent2[point:]
    
    def evaluate_fitness_parallel(self, individual):
        return self.fitness(individual)

    def run_evolution(self):
        # for generation in range(self.generations):
        generation = 0
        pbar = tqdm()
        with ThreadPoolExecutor() as executor:
            while True:
                
                fitness_scores = []
                for individual in self.population:
                    f = self.fitness(individual)
                    fitness_scores.append(f)
                    if int(f) == 1:
                        print("Solution found at generation", generation)
                        print("Fitness calls:", self.fitness.calls)
                        return


                selected_parents=choices(self.population, k=self.population_size,weights=fitness_scores)

                # Create a new population through crossover and mutation
                new_population = []
                for i in range(self.population_size):
                    # Parent selection
                    parent1 = choice(selected_parents)
                    parent2 = choice(selected_parents)

                    # Crossover and mutation
                    child = self.mutate0(self.crossover(parent1, parent2))

                    new_population.append(child)

                self.population = new_population
                generation += 1
                pbar.update()

                if generation % 1000 == 0:
                    print(f"gen {generation} best fitness: {max(fitness_scores)}")

                if generation > 1e6:
                    print("no solution found")
                    print("Best fitness:", max(fitness_scores))
                    print("Fitness calls:", self.fitness.calls)
                    return

## Istance 1


In [ ]:
fitness = lab9_lib.make_problem(1)

ga = GA(
    fitness,
    population_size=100,
    number_generations=200,
    mutation_rate=0.01,
    genome_size=100,
    parent_selection_size=10,
)
ga.run_evolution()

# 100 -> 30it
# 200 -> 120it


## Istance 2


In [ ]:
fitness = lab9_lib.make_problem(2)

ga = GA(
    fitness,
    population_size=100,
    number_generations=200,
    mutation_rate=0.01,
    genome_size=200,
    parent_selection_size=10,
)
ga.run_evolution()

## Istance 5

In [ ]:
fitness = lab9_lib.make_problem(5)

ga = GA(
    fitness,
    population_size=100,
    number_generations=200,
    mutation_rate=0.01,
    genome_size=200,
    parent_selection_size=10,
)
ga.run_evolution()

## Istance 10


In [ ]:
fitness = lab9_lib.make_problem(10)

ga = GA(
    fitness,
    population_size=100,
    number_generations=200,
    mutation_rate=0.01,
    genome_size=100,
    parent_selection_size=10,
)
ga.run_evolution()

# Hill Climbing

In [ ]:
class HillClimb:
    def __init__(
        self,
        fitness,
        instance=1,
        max_iterations=50000,
        neighbourhood_size=10,
        genome_size=1000,
        step_size=1,
    ):
        self.max_iterations = max_iterations
        self.neighbourhood_size = neighbourhood_size
        self.genome_size = genome_size
        self.step_size = step_size  # number of bits to flip
        self.fitness = fitness
        self.progenitor = choices([0, 1], k=genome_size)
        self.instance = instance

    def generate_neighbour(self, individual):
        neighbours = []

        for _ in range(self.neighbourhood_size):
            neighbour = individual.copy()
            for i in range(self.step_size):
                index = randint(0, self.genome_size - 1)
                neighbour[index] = neighbour[index] ^ 1
            neighbours.append(neighbour)

        return neighbours
    

    def climb(self):
        iter = 0
        best = self.progenitor
        best_fitness = self.fitness(best)

        saturation = 0
        saturation_limit = 1000

        while best_fitness < 1.0:

            if iter % 1000 == 0:
                print(f"iter {iter} best fitness: {best_fitness}, sum: {sum(best)}")

            neighbours = self.generate_neighbour(best)
            fitnesses = [self.fitness(n) for n in neighbours]
            # fitnesses = [sum(n) for n in neighbours]  
            

            new_best_fitness = max(fitnesses)
            if new_best_fitness > best_fitness:
                best_fitness = new_best_fitness
                best = neighbours[fitnesses.index(new_best_fitness)]
                saturation = 0
            else:
                saturation += 1
                if saturation >= saturation_limit and self.neighbourhood_size <= self.genome_size // 2:
                    self.neighbourhood_size *= 2
                    print(" saturation, doubling neighbourhood size to", self.neighbourhood_size ,"maxed"if self.neighbourhood_size * 2 > self.genome_size else "")
                    saturation = 0

            if iter > self.max_iterations:
                print("no solution found")
                print("Best fitness:", best_fitness)
                print("Fitness calls:", self.fitness.calls)
                return
            
            iter += 1

        print(f"\nSolution found at iter {iter}")
        print("Fitness calls:", self.fitness.calls)
        print("Check:", "OK" if sum(best) == self.genome_size else "FAIL")
        return

In [ ]:
class HillClimb2:
    def __init__(
        self,
        fitness,
        instance=1,
        max_iterations=50000,
        neighbourhood_size=10,
        genome_size=1000,
        step_size=1,
    ):
        self.max_iterations = max_iterations
        self.neighbourhood_size = neighbourhood_size
        self.genome_size = genome_size
        self.step_size = step_size  # number of bits to flip
        self.fitness = fitness
        self.progenitor = choices([0, 1], k=genome_size)
        self.instance = instance

    def generate_neighbour(self, individual):
        neighbours = []

        for _ in range(self.neighbourhood_size):
            neighbour = individual.copy()

            for i in range(self.step_size):
                index = randint(0, self.genome_size - 1)
                neighbour[index] = neighbour[index] ^ 1

            shuffle(neighbour)
            neighbours.append(neighbour)

        return neighbours
    

    def climb(self):
        iter = 0
        best = self.progenitor
        best_fitness = self.fitness(best)

        saturation = 0
        saturation_limit = 1000

        while best_fitness < 1.0:

            if iter % 1000 == 0:
                print(f"iter {iter} best fitness: {best_fitness}, sum: {sum(best)}")

            neighbours = self.generate_neighbour(best)
            fitnesses = [self.fitness(n) for n in neighbours]
  
            

            new_best_fitness = max(fitnesses)
            if new_best_fitness >= best_fitness:
                best_fitness = new_best_fitness
                best = neighbours[fitnesses.index(new_best_fitness)]
                saturation = 0
            else:
                saturation += 1
                if saturation >= saturation_limit and self.neighbourhood_size <= self.genome_size // 2:
                    self.neighbourhood_size *= 2
                    print(" saturation, doubling neighbourhood size to", self.neighbourhood_size ,"maxed"if self.neighbourhood_size * 2 > self.genome_size else "")
                    saturation = 0

            # if iter > self.max_iterations:
            #     print("no solution found")
            #     print("Best fitness:", best_fitness)
            #     print("Fitness calls:", self.fitness.calls)
            #     return
            
            iter += 1

        print(f"\nSolution found at iter {iter}")
        print("Fitness calls:", self.fitness.calls)
        print("Check:", "OK" if sum(best) == self.genome_size else "FAIL")
        return

In [ ]:
# Instance 1
fitness = lab9_lib.make_problem(1)

genome_size = 1000
hill_climb = HillClimb2(fitness,instance=1,genome_size=genome_size,max_iterations=50000,neighbourhood_size=1,step_size=1)
hill_climb.climb()

In [ ]:
# Instance 2
fitness = lab9_lib.make_problem(2)

genome_size = 1000
hill_climb = HillClimb2(fitness,instance=2,genome_size=genome_size,max_iterations=100000,neighbourhood_size=2,step_size=2)
hill_climb.climb()

In [ ]:
# Instance 5
fitness = lab9_lib.make_problem(5)

genome_size = 1000
hill_climb = HillClimb(fitness,instance=5,genome_size=genome_size,max_iterations=50000,neighbourhood_size=1,step_size=1)
hill_climb.climb()

In [ ]:
# Instance 10
fitness = lab9_lib.make_problem(10)

# genome_size = 1000
# hill_climb = HillClimb(fitness,instance=10,genome_size=genome_size,max_iterations=50000,neighbourhood_size=1,step_size=1)
# hill_climb.climb()